In [ ]:
# imports - ignore
# !pip install gymnasium stable_baselines3[extra] opencv-python highway_env

In [1]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
from stable_baselines3 import DQN

import highway_env  # noqa: F401

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
objc[50419]: Class SDLApplication is implemented in both /Users/dkoffical/envs/myenv312/lib/python3.12/site-packages/cv2/.dylibs/libSDL2-2.0.0.dylib (0x2bf490890) and /Users/dkoffical/envs/myenv312/lib/python3.12/site-packages/pygame/.dylibs/libSDL2-2.0.0.dylib (0x302e612c8). One of the two will be used. Which one is undefined.
objc[50419]: Class SDLAppDelegate is implemented in both /Users/dkoffical/envs/myenv312/lib/python3.12/site-packages/cv2/.dylibs/libSDL2-2.0.0.dylib (0x2bf4908e0) and /Users/dkoffical/envs/myenv312/lib/python3.12/site-packages/pygame/.dylibs/libSDL2-2.0.0.dylib (0x302e61318). One of the two will be used. W

In [29]:
TRAIN = False

# Create the environment
env = gym.make("roundabout-v0", render_mode="rgb_array")

# add disturbances



obs, info = env.reset()

# Create the model
model = DQN(
    "MlpPolicy",
    env,
    policy_kwargs=dict(net_arch=[256, 256]),
    learning_rate=5e-4,
    buffer_size=15000,
    learning_starts=200,
    batch_size=32,
    gamma=0.8,
    train_freq=1,
    gradient_steps=1,
    target_update_interval=50,
    verbose=1,
    tensorboard_log="roundabout_dqn/",
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [5]:
import numpy as np
print(np.__version__)

2.0.2


In [3]:
# Train the model
if TRAIN:
    model.learn(total_timesteps=int(2e4))
    model.save("roundabout_dqn/model")
    del model

In [7]:
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
# # Change working directory
# import os
# os.chdir('/content/drive/MyDrive/CS238V-Cars/')  # put your folder path here

# # Check current working directory
# !pwd

/content/drive/MyDrive/CS238V-Cars


In [ ]:
# Actuator fuzzing

import numpy as np

def low_level_agent_fuzzing(vehicle, acc_std=0.5, steer_std=0.1, max_steer_std=0.05):
    """Add Gaussian noise to the low-level controllers of a vehicle.

    Args:
        vehicle: ControlledVehicle or MDPVehicle instance
        acc_std: std deviation for acceleration noise [m/s²]
        steer_std: std deviation for steering noise [rad]
    """

    # Backup original functions
    original_speed_control = vehicle.speed_control
    original_steering_control = vehicle.steering_control
    
    # Optional: perturb MAX_STEERING_ANGLE once per vehicle
    vehicle.MAX_STEERING_ANGLE = max(
        0.05,  # avoid zero max steering
        vehicle.MAX_STEERING_ANGLE + np.random.normal(0, max_steer_std)
    )

    # Redefine speed_control with Gaussian noise
    def noisy_speed_control(target_speed):
        base = original_speed_control(target_speed)
        noise = np.random.normal(0, acc_std)
        return base + noise

    # Redefine steering_control with Gaussian noise
    def noisy_steering_control(target_lane_index):
        base = original_steering_control(target_lane_index)
        noise = np.random.normal(0, steer_std)
        return base + noise

    vehicle.speed_control = noisy_speed_control
    vehicle.steering_control = noisy_steering_control


In [ ]:
import random
# Run the trained model and record video
model = DQN.load("roundabout_dqn/model", env=env, device="cpu")
# model.set_env(env)
env = RecordVideo(
    env, video_folder="roundabout_dqn/videos/action_controller_fuzz", episode_trigger=lambda e: True
)
env.unwrapped.config["simulation_frequency"] = 15  # Higher FPS for rendering
env.unwrapped.config.update({
     "observation": {
         # default
                    "type": "Kinematics",
                    "absolute": True,
                    "features": ["presence", "x", "y", "vx", "vy", "cos_h", "sin_h"],
                    "features_range": {
                        "x": [-100, 100],
                        "y": [-100, 100],
                        "vx": [-15, 15], #[-15, 15],
                        "vy": [-15, 15], #[-15, 15],
                    },
                    
                    # # add new
                    "vehicles_count": 5,
                    # "order": "shuffled",
                    
                },
     
     
    # "sensor_noise": 0.02
})
low_level_agent_fuzzing(env.unwrapped.vehicle, acc_std=0.0, steer_std=5)

env.unwrapped.set_record_video_wrapper(env)

for videos in range(2):
    done = truncated = False
    obs, info = env.reset()
    while not (done or truncated):
        # Predict
        action, _states = model.predict(obs, deterministic=True)
        
        # # action fuzz
        # if random.random() < 0.4:
        #     available = env.unwrapped.action_type.get_available_actions()
        #     print(available)
        #     action = random.choice(available)

        # # Get reward
        obs, reward, done, truncated, info = env.step(action)
        
        # Render
        env.render()
    env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/dkoffical/envs/myenv312/lib/python3.12/site-packages/gymnasium/wrappers/rendering.py:293: UserWarning: WARN: Overwriting existing videos at /Users/dkoffical/Documents/GitHub/StanfordRoundabout/roundabout_dqn/videos/action_controller_fuzz folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [ ]:
# !pip freeze > colab_requirements.txt

In [ ]:
# imports
# # import numpy
# # import gymnasium
# # import stable_baselines3
# # import cv2
# # import highway_env
# import torch
# import torchvision
# import torchaudio


# # print("numpy:", numpy.__version__)
# # print("gymnasium:", gymnasium.__version__)
# # print("stable_baselines3:", stable_baselines3.__version__)
# # print("opencv-python:", cv2.__version__)
# # print("highway_env:", highway_env.__version__)

# print("torch:", torch.__version__)
# print("torchvision:", torchvision.__version__)
# print("torchaudio:", torchaudio.__version__)


numpy: 2.0.2
gymnasium: 1.2.3
stable_baselines3: 2.7.1
opencv-python: 4.13.0
highway_env: 1.10.2
